# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

![heartimage](https://2rdnmg1qbg403gumla1v9i2h-wpengine.netdna-ssl.com/wp-content/uploads/sites/3/2019/10/cardiacDocs-1125401691-770x553-650x428.jpg)

1. [Problem Definition](#problem_definition)

2. [Importing Libraries](#import)

3. [Data Dictionary](#data_dictionary)

4. [Data Exploration](#data_exploration)

    4.1 [Heart Disease frequency according to Sex](#exploration_by_sex)

    4.2 [Heart Disease frequency for chest pain](#exploration_by_pain)

    4.3 [Correlation matrix](#correlation)
    
5. [Model Building](#model_building)

6. [HyperParameter Tuning](#hyperparameter_tuning)
    
    6.1 [Manual Hyperparameter Tuning](#manual_tuning)
    
    6.2 [RandomizedSearchCV](#random_tuning)
    
    6.3 [GridSearchCV](#grid_tuning)

7. [Evaluation beyond accuracy](#evaluation)

    7.1. [Different Evaluation metrics using Cross-Validation](#cross_validation)

9. [Feature Importance](#feature_importance)

<a id = "problem_definition"></a>
# Problem Definition

Given clinical parameters about a patient, can we predict whether or not they have heart disease?

<a id = "import"></a>
# Importing Libraries

In [ ]:
# importing all the necessary frame works

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix , classification_report , precision_score,recall_score,f1_score,plot_roc_curve

from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV , GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/heart-disease-uci/heart.csv')

<a id = "data_dictionary"></a>
# Data Dictionary

1. `age` - age in years
2. `sex` - (1 = male; 0 = female)
3. `cp` - chest pain 
        type 0: Typical angina: chest pain related decrease blood supply to the heart 
        type 1: Atypical angina: chest pain not related to heart 
        type 2: Non-anginal pain: typically esophageal spasms (non heart related) 
        type 3: Asymptomatic: chest pain not showing signs of disease
4. `trestbps` - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
5. `chol` - serum cholestoral in mg/dl
            * serum = LDL + HDL + .2 * triglycerides
            * above 200 is cause for concern
6. `fbs` - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
            * '>126' mg/dL signals diabetes
7. `restecg` - resting electrocardiographic results 
        * 0: Nothing to note 
        * 1: ST-T Wave abnormality
            - can range from mild symptoms to severe problems
            - signals non-normal heart beat
        * 2: Possible or definite left ventricular hypertrophy
            - Enlarged heart's main pumping chamber
8. `thalach` - maximum heart rate achieved
9. `exang` - exercise induced angina (1 = yes; 0 = no)
10. `oldpeak` - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more
11. `slope` - the slope of the peak exercise ST segment 
        * 0: Upsloping: better heart rate with excercise (uncommon) 
        * 1: Flatsloping: minimal change (typical healthy heart) 
        * 2: Downslopins: signs of unhealthy heart
12. `ca` - number of major vessels (0-3) colored by flourosopy 
        * colored vessel means the doctor can see the blood passing through
        * the more blood movement the better (no clots)
13. `thal` - thalium stress result
        * 1,3: normal
        * 6: fixed defect: used to be defect but ok now
        * 7: reversable defect: no proper blood movement when excercising
14. `target` - have disease or not (1=yes, 0=no) (the predicted attribute)

In [ ]:
df.head()

<a id = "data_exploration"></a>
## Data Exploration

1. What questions are you trying to solve 
2. What kind of data do we have and how we handle it
3. What is missing and how you are going to handle it
4. Where are the outliers and why should we care about them
5. How can you add, change or remove features to get more out of your data


In [ ]:
df.target.value_counts()

In [ ]:
df['target'].value_counts().plot(kind='bar');

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

<a id = "exploration_by_sex"></a>
### Heart disease frequency according to sex

In [ ]:
df.sex.value_counts()

In [ ]:
 pd.crosstab(df.target,df.sex)

 In this,

    total female = 96
    affected female = 72 , one third of the female are affected
    
    total male = 207
    affected male = 93 half of the male are affected 

In [ ]:
pd.crosstab(df.target,df.sex).plot(kind = "bar",color = ['salmon','lightblue'],figsize=(10,6));
    
plt.title("Heart diesase with gender")
plt.legend(['female','male'])
plt.xlabel("0 = No heart disease  1 = heart disease")

In [ ]:
plt.figure(figsize=(10,7))

plt.scatter(df.age[df.target==1],
            df.thalach[df.target==1],
            c = 'darkred')

plt.scatter(df.age[df.target==0],
            df.thalach[df.target==0],
            c = 'salmon')

plt.title("Heart disease in function of age and max heart rate")
plt.xlabel('Age')
plt.ylabel('Max heart rate (thalach)')
plt.legend(['Disease','No disease'])

In [ ]:
## check the distribution of the age column with hist
plt.hist(df.age)

<a id = "exploration_by_pain"></a>
### Heart disease frequency for chest pain

In [ ]:
pd.crosstab(df.cp,df.target)

In [ ]:
pd.crosstab(df.cp,df.target).plot(kind='bar',color=['salmon','darkred'])


plt.title("Heart disease in function chest pain with target")
plt.xlabel('Chest pain')
plt.ylabel('Amount')
plt.legend(['NO disease','Disease'])

<a id = "correlation"></a>
### Correlation matrix

In [ ]:
df.corr()

In [ ]:
corr_mat = df.corr()

plt.figure(figsize=(15,10))
sns.heatmap(corr_mat,annot=True,linewidths=0.5)

<a id = "model_building"></a>
## Model Building

In [ ]:
# split data into X and y

X = df.drop('target',axis=1)
y = df['target']

In [ ]:
# train test spliting

np.random.seed(42) #To reproduced the randomized data again

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2)

X_train.shape , X_test.shape

**Models to build**

1. RandomForestClassifier
2. Logistic Regression
3. KNearest neighbor classifier

In [ ]:
# Time to build a machine learning model

models = {'logistic regression':LogisticRegression(),
          'Random forest classifier':RandomForestClassifier(),
          'KNearest neighbor':KNeighborsClassifier()
         }

np.random.seed(42)

def fit_and_score(models,X_train,X_test,y_train,y_test):
    """
    Fits and evaluatest different machine learning models.
    models : a dict of different sciit learn machine learning models.
    X_train : training data (no labels)
    X_test : Testing data (no labels)
    y_train : training labels
    y_test : testing labels    
    """
    scores = {}
    
    for name , model in models.items():
        model.fit(X_train,y_train)
        scores[name] = model.score(X_test,y_test)
        
    return scores
        

In [ ]:
model_scores = fit_and_score(models=models,X_train=X_train,X_test=X_test,
                             y_train=y_train,y_test=y_test)

model_scores

In [ ]:
model_scores_df = pd.DataFrame(model_scores,index=['accuracy'])
model_scores_df.T.plot.bar();


Now we've got a baseline model... and we know a model's first predictions aren't always what we should based our next steps off. What should we do?

Let's look at the following:

    - Hypyterparameter tuning
    - Feature importance
    - Confusion matrix
    - Cross-validation
    - Precision
    - Recall
    - F1 score
    - Classification report
    - ROC curve
    - Area under the curve (AUC)

<a id = "hyperparameter_tuning"></a>
## Hyper Parameter Tuning


<a id = "manual_tuning"></a>
### Manual HyperParameter Tuning

In [ ]:
#Tuning knn

train_scores = []
test_scores = []

neighbors = range(1,21)

knn = KNeighborsClassifier()

for i in neighbors:
    knn.set_params(n_neighbors = i)
    
    knn.fit(X_train,y_train)
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [ ]:
plt.plot(neighbors,train_scores,label = 'train scores')
plt.plot(neighbors,test_scores,label = 'train scores')
plt.legend()
plt.xlabel("Number of neighbors")
plt.ylabel("Model score")
plt.xticks(np.arange(1,21,1));
print(f"Maximum KNN score on the test data: {max(test_scores)*100:.2f}%")


<a id = "random_tuning"></a>
### Hyperparameter tuning with Randomized search CV

We're going to tune , 

 - LogisticRegression()
 - RandomForestClassifier()

In [ ]:
#logistic regression grid
log_reg_grid = {'C': np.logspace(-4,4,20),
                'solver': ['liblinear']}

#random forest grid
rf_grid = {'n_estimators':np.arange(10,1000,50),
           'max_depth': [None,3,5,10],
           'min_samples_split':np.arange(2,20,2),
           'min_samples_leaf':np.arange(1,20,2)}


In [ ]:
np.random.seed(42)

rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

rs_log_reg.fit(X_train,y_train)

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_log_reg.score(X_test,y_test)

In [ ]:
# Setup random seed
np.random.seed(42)

# Setup random hyperparameter search for RandomForestClassifier
rs_rf = RandomizedSearchCV(RandomForestClassifier(), 
                           param_distributions=rf_grid,
                           cv=5,
                           n_iter=20,
                           verbose=True)

# Fit random hyperparameter search model for RandomForestClassifier()
rs_rf.fit(X_train, y_train)

In [ ]:
rs_rf.best_params_

In [ ]:
rs_rf.score(X_test,y_test)

<a id = "grid_tuning"></a>
### Hyperparameter tuning with GridsearchCV

In [ ]:
#Different hyperprameters for our logistic regression model

log_reg_grid = {'C': np.logspace(-4, 4, 30),
                "solver": ["liblinear"]}

gs_log_reg = GridSearchCV(LogisticRegression(),
                         param_grid=log_reg_grid,
                         cv=5,
                         verbose=True)

gs_log_reg.fit(X_train,y_train);

In [ ]:
gs_log_reg.best_params_


In [ ]:
gs_log_reg.score(X_test,y_test)

<a id = "evaluation"></a>
### Evaluting our tuned machine learning classifier, beyond accuracy

- ROC curve and AUC score
- Confusion matrix
- Classification report
- Precision
- Recall
- F1-score

In [ ]:
y_preds = gs_log_reg.predict(X_test)

In [ ]:
y_preds

In [ ]:
plot_roc_curve(gs_log_reg, X_test, y_test)

In [ ]:
sns.set(font_scale=1.5)

sns.heatmap(confusion_matrix(y_test,y_preds),
            annot=True,
            cbar=False)

Now we've got a ROC curve, an AUC metric and a confusion matrix, let's get a classification report as well as cross-validated precision, recall and f1-score.

In [ ]:
print(classification_report(y_test,y_preds))

<a id = "cross_validation"></a>
### Different evaluation metrics using cross-validation

We're going to calculate accuracy, precision, recall and f1-score of our model using cross-validation and to do so we'll be using cross_val_score().

In [ ]:
# Check best hyperparameters
gs_log_reg.best_params_

In [ ]:
# Create a new classifier with best parameters
clf = LogisticRegression(C=0.20433597178569418,
                         solver="liblinear")

In [ ]:
# Cross-validated accuracy
cv_acc = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring="accuracy")
cv_acc = cv_acc.mean()
print(f"Cross Validated accuracy {cv_acc}")

In [ ]:
# Cross-validated precision
cv_precision = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring="precision")
cv_precision=np.mean(cv_precision)
print(f"Cross Validated Precision {cv_precision}")

In [ ]:
# Cross-validated recall
cv_recall = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring="recall")
cv_recall=np.mean(cv_recall)
print(f"Cross Validated Recall {cv_recall}")

In [ ]:
# Cross-validated f1_score
cv_f1 = cross_val_score(clf,
                         X,
                         y,
                         cv=5,
                         scoring="f1")
cv_f1=np.mean(cv_f1)
print(f"Cross Validated F1_score {cv_f1}")

In [ ]:
# Visualizing cross-validated metrics
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                           "Precision": cv_precision,
                           "Recall": cv_recall,
                           "F1": cv_f1},
                          index=[0])

cv_metrics.T.plot.bar(title="Cross-validated classification metrics",
                      legend=False);

<a id = "feature_importance"></a>
## Feature Importance

Feature importance is another as asking, "which features contributed most to the outcomes of the model and how did they contribute?"

Finding feature importance is different for each machine learning model. One way to find feature importance is to search for "(MODEL NAME) feature importance".

Let's find the feature importance for our LogisticRegression model...

In [ ]:

# Fit an instance of LogisticRegression
clf = LogisticRegression(C=0.20433597178569418,
                         solver="liblinear")

clf.fit(X_train, y_train);

For logistic regression coef_ is used to find the feature importances of the model

In [ ]:
# Check coef_
clf.coef_

In [ ]:
# Match coef's of features to columns
feature_dict = dict(zip(df.columns, list(clf.coef_[0])))
feature_dict

In [ ]:
# Visualize feature importance
feature_df = pd.DataFrame(feature_dict, index=[0])
feature_df.T.plot.bar(title="Feature Importance", legend=False);

Sex is highly negative correlated with the target variable lets look at it..

In [ ]:
pd.crosstab(df["sex"], df["target"])

Sex class is highly imbalanced with lower female and high male values and also we can see that ratio of the female is 3 : 1 and for the male it is more or less 1:2 hence as the sex is increasing the target value is decreasing

In [ ]:
pd.crosstab(df["slope"], df["target"])

slope - the slope of the peak exercise ST segment

- 0: Upsloping: better heart rate with excercise (uncommon)
- 1: Flatsloping: minimal change (typical healthy heart)
- 2: Downslopins: signs of unhealthy heart

# Please do Upvote if you find it useful